In [5]:
import sys
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
import shutil
import torch
import subprocess

# Tambahkan path utils jika dijalankan dari notebooks/
if os.path.basename(os.getcwd()) == 'notebooks':
    sys.path.append('../')

from utils.functions import is_custom_model

# Setup path
repo_path = os.getcwd()
repo_path = os.path.dirname(repo_path)
yolov11_path = os.path.join(repo_path, 'yolov11')  # kalau kamu menyimpan YAML/data di folder ini

# Deteksi GPU/CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device_str = "0" if torch.cuda.is_available() else "cpu"

In [5]:
%pip install -U ultralytics

  Using cached ultralytics-8.3.151-py3-none-any.whl.metadata (37 kB)
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
Using cached ultralytics-8.3.151-py3-none-any.whl (1.0 MB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)
Using cached ultralytics_thop-2.0.14-py3-none-any.whl (26 kB)
Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)

   ---------- ----------------------------- 1/4 [opencv-python]
   ---------- ----------------------------- 1/4 [opencv-python]
   ---------- ----------------------------- 1/4 [opencv-python]
   ------------------------------ --------- 3/4 [ultralytics]
   ------------------------------ --------- 3/4 [ultralytics]
   ------------------------------ --------- 3/4 [ultralytics]
   ------------------------------ --------- 3/4 [ultralytics]
   ----------------

In [6]:
import torch
torch.cuda.is_available()
print(torch.cuda.get_device_name(0)) 

NVIDIA GeForce RTX 3070 Ti Laptop GPU


In [4]:
# %pip install -q ultralytics
!python "{repo_path}/utils/yolov11startE.py" --reporoot "{repo_path}"

Repo path: c:\Users\Evaldo Raynardi\OneDrive\Desktop\Kuilah\Semester 4\Machine Learning\testing\YOLO-Parking-Spot-Rombak
Yolo path: c:\Users\Evaldo Raynardi\OneDrive\Desktop\Kuilah\Semester 4\Machine Learning\testing\YOLO-Parking-Spot-Rombak\yolov11
Data copied to c:\Users\Evaldo Raynardi\OneDrive\Desktop\Kuilah\Semester 4\Machine Learning\testing\YOLO-Parking-Spot-Rombak\yolov11\data ✅


In [5]:
%cd $yolov11_path

c:\Users\Evaldo Raynardi\OneDrive\Desktop\Kuilah\Semester 4\Machine Learning\testing\YOLO-Parking-Spot-Rombak\yolov11


c:\Users\Evaldo Raynardi\anaconda3\envs\machineLearning\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
print(f"{yolov11_path}/dataset.yaml")

c:\Users\Evaldo Raynardi\OneDrive\Desktop\Kuilah\Semester 4\Machine Learning\testing\YOLO-Parking-Spot-Rombak\yolov11/dataset.yaml


In [16]:
def yolov11_detect_trains(model, device="cpu"):
    model, model_path = is_custom_model(model, "11")
    data_yaml = f"{yolov11_path}/dataset.yaml"
    project_dir = f"{repo_path}/models/{model[:-3]}"
    log_file = f"{project_dir}/training_log.txt"

    command = [
        "yolo", "task=detect", "mode=train",
        f"model={model_path}",
        "imgsz=640",
        f"data={data_yaml}",
        "epochs=10",
        "batch=16",
        f"project={project_dir}",
        f"device={device}"
    ]

    os.makedirs(project_dir, exist_ok=True)

    with open(log_file, "w") as f:
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        for line in process.stdout:
            print(line, end='')
            f.write(line)


In [18]:
def yolov11_detect_predicts(model, device="cpu", conf=0.25, iou=0.45, source_folder="val/images"):
    model, model_path = is_custom_model(model, "11")
    model_name = model[:-3]
    result_dir = os.path.join(repo_path, f"results/{model_name}/")

    # Hapus folder hasil sebelumnya
    if os.path.exists(result_dir):
        shutil.rmtree(result_dir)

    source_path = os.path.join(repo_path, "data", source_folder)

    if not os.path.exists(source_path):
        raise FileNotFoundError(f"Source folder {source_path} tidak ditemukan!")

    command = [
        "yolo", "task=detect", "mode=predict",
        f"model={model_path}",
        f"source={source_path}",
        "save_txt=True",
        "exist_ok=True",
        "imgsz=640",
        f"conf={conf}",
        f"iou={iou}",
        f"project={os.path.join(repo_path, 'results')}",
        f"name={model_name}/yolo_images",
        f"device={device}"
    ]

    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    for line in process.stdout:
        print(line, end='')

    # Pindahkan folder label
    label_src = os.path.join(repo_path, f'results/{model_name}/yolo_images/labels')
    label_dst = os.path.join(repo_path, f'results/{model_name}/labels')
    if os.path.exists(label_src):
        shutil.move(label_src, label_dst)


In [17]:
# name:
# yolov8_detect_trains('yolov8n.pt', device)
yolov11_detect_trains('yolo11n.pt', device=device_str) # GPU

Ultralytics 8.3.151  Python-3.12.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\Users\Evaldo Raynardi\OneDrive\Desktop\Kuilah\Semester 4\Machine Learning\testing\YOLO-Parking-Spot-Rombak\yolov11/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=c:\Users\Evaldo Raynardi\OneDrive\Desktop\Kuilah\Semester 4\Machine Learning\testing\YOLO-Pa

In [20]:
yolov11_detect_predicts(f'{repo_path}/models/yolo11n/train/weights/best.pt', device=device_str)

# uploaded model:
#yolov8_detect_predict('yolov8n_fold_0.pt', device, conf=0.4, iou=0.35)

Ultralytics 8.3.151  Python-3.12.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs

image 1/321 c:\Users\Evaldo Raynardi\OneDrive\Desktop\Kuilah\Semester 4\Machine Learning\testing\YOLO-Parking-Spot-Rombak\data\val\images\2012-09-11_15_36_32_jpg.rf.b2b1767cafabff6378468d0d71c1ecd2.jpg: 640x640 32 emptys, 70 filleds, 5.0ms
image 2/321 c:\Users\Evaldo Raynardi\OneDrive\Desktop\Kuilah\Semester 4\Machine Learning\testing\YOLO-Parking-Spot-Rombak\data\val\images\2012-09-11_15_45_57_jpg.rf.01d1052a2938ca83e8bdb05e2f86c6b4.jpg: 640x640 31 emptys, 71 filleds, 4.9ms
image 3/321 c:\Users\Evaldo Raynardi\OneDrive\Desktop\Kuilah\Semester 4\Machine Learning\testing\YOLO-Parking-Spot-Rombak\data\val\images\2012-09-11_17_12_07_jpg.rf.e73a27714868ec27c08e1960bfa4130f.jpg: 640x640 25 emptys, 76 filleds, 5.0ms
image 4/321 c:\Users\Evaldo Raynardi\OneDrive\Desktop\Kuilah\Semester 4\Machine Learnin